In [1]:
from pathlib import Path
from io import StringIO
import pandas as pd
import numpy as np

In [2]:
grid, moves = Path("data/15sample1.txt").read_text().split("\n\n")
grid = (
    pd.read_csv(StringIO(grid), engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)
grid1 = grid.copy()
moves = [x for l in moves.split("\n") for x in l]
dirs = {"<": (0, -1), ">": (0, 1), "^": (-1, 0), "v": (1, 0)}

In [3]:
def find_first(line, ch):
    try:
        i = next(i for i, x in enumerate(line) if x == ch)
    except StopIteration:
        return None
    return i


def pprint(grid):
    print("\n".join("".join(l) for l in wide_grid))

In [4]:
def move(x, y, dir):
    dx, dy = dirs[dir]
    if dir == "<":
        line = grid[x, 1:y][::-1]
    elif dir == ">":
        line = grid[x, (y + 1) : -1]
    elif dir == "^":
        line = grid[1:x, y][::-1]
    elif dir == "v":
        line = grid[(x + 1) : -1, y]

    if (early := find_first(line, "#")) is not None:
        line = line[:early]

    if len(line) == 0:
        return x, y

    i = find_first(line, ".")
    if i is None:
        return x, y

    line[1 : i + 1] = line[:i]
    line[0] = "@"
    grid[x, y] = "."
    return x + dx, y + dy

In [5]:
x, y = np.where(grid == "@")[0][0], np.where(grid == "@")[1][0]
for dir in moves:
    # pprint(grid)
    x, y = move(x, y, dir)
    # print(dir)
# pprint(grid)
sum(i * 100 + j for (i, j), v in np.ndenumerate(grid) if v == "O")

10092

In [6]:
b = StringIO()
transform = {"#": "##", "O": "[]", ".": "..", "@": "@."}

for line in grid1:
    for x in line:
        b.write(transform[x])
    b.write("\n")
b.seek(0)

wide_grid = (
    pd.read_csv(b, engine="python", sep=r"\s*", names=range(-1, 1000))
    .dropna(how="all", axis=1)
    .values
)

In [7]:
pprint(wide_grid)

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
